Como os cálculos para este tipo de exercícios são algo extensivos e podem dar aso a erro, utilizámos python para computar a entropia e o ganho de informação para um dado conjunto de dados.  
Carregamos os dados desta forma. Se quisermos tirar ou adicionar instâncias é só comentar/descomentar as linhas onde figuram.  
A classe (yout) é a primeira (índice 0).

In [37]:
data  = []
# data += [["C", 0.22, 2, 0, 1]]
# data += [["B", 0.06, 0, 0, 0]]
# data += [["C", 0.16, 1, 2, 2]]
# data += [["B", 0.21, 0, 0, 0]]
data += [["C", 0.01, 2, 2, 0]]
data += [["B", 0.3 , 0, 1, 0]]
data += [["A", 0.76, 0, 1, 1]]
data += [["A", 0.86, 1, 0, 0]]
data += [["C", 0.93, 0, 1, 1]]
data += [["C", 0.47, 0, 1, 1]]
data += [["A", 0.73, 1, 0, 0]]
data += [["B", 0.89, 1, 2, 0]]

Calculamos a entropia da variável yi como se segue:  

In [40]:
import math

def entropy(data, i):
    
    hist = []
    for instance in data:
        found = False
        for pair in hist:
            value = pair[0]
            count = pair[1]
            if instance[i] == value:
                pair[1] += 1
                found = True
        if not found:
            hist += [[instance[i], 1]]

    total = len(data)

    entr = 0
    for pair in hist:
        entr -= pair[1]/total * math.log(pair[1]/total, 2)

    return entr

Calculamos a entropia da variável yi em relação à variável ycond da seguinte forma: 

In [43]:
def entropy_cond(data, i, cond):

    hist = []
    for j in range(len(data)):
        found = False
        for pair in hist:
            value = pair[0]
            count = pair[1]
            if data[j][cond] == value:
                pair[1] += [j]
                found = True
        if not found:
            hist += [[data[j][cond], [j]]]
    
    total = len(data)

    entr = 0
    for pair in hist:
        data_subset = []
        for j in pair[1]:
            data_subset += [data[j]]
        entr += len(pair[1])/total * entropy(data_subset, i)

    return entr

Já temos tudo o que é preciso para calcular o ganho de informação da variável yi.  
Como variáveis discretas e contínuas se calculam de maneiras diferentes, temos duas versões.  
Em information_gain_cont tem-se em conta todos os modos de dividir a variável contínua em dois grupos e escolhe-se o modo com maior ganho de informação.

In [46]:
def information_gain(data, i):
    return entropy(data, 0) - entropy_cond(data, 0, i)

def information_gain_cont(data, i):
    
    max_ig = 0
    cut = 0
    data = sorted(data, key=lambda x: x[i])
    for j in range(1, len(data)):
        if data[j - 1][i] != data[j][i]:
            data_subset1 = data[:j]
            data_subset2 = data[j:]
            entr = j / len(data) * entropy(data_subset1, 0) + (len(data) - j) / len(data) * entropy(data_subset2, 0)
            ig = entropy(data, 0) - entr
            if ig > max_ig:
                max_ig = ig
                cut = (data[j][i] + data[j - 1][i]) / 2

    return max_ig, cut

Agora basta chamar as funções e observar os resultados.

In [49]:
print(f"IG(y{1}) = {round(information_gain_cont(data, 1)[0], 3), information_gain_cont(data, 1)[1]}")
for i in [2, 3, 4]:
    print(f"IG(y{i}) = {round(information_gain(data, i), 3)}")

IG(y1) = (0.36, 0.6)
IG(y2) = 0.467
IG(y3) = 0.561
IG(y4) = 0.266


Como y3 tem ganho de informação maior, escolhe-se a variável y3.  
Observemos o subconjunto de dados depois da escolha y3 = 1.

In [78]:
data  = []
# data += [["C", 0.22, 2, 0, 1]]
# data += [["B", 0.06, 0, 0, 0]]
# data += [["C", 0.16, 1, 2, 2]]
# data += [["B", 0.21, 0, 0, 0]]
#data += [["C", 0.01, 2, 2, 0]]
data += [["B", 0.3 , 0, 1, 0]]
data += [["A", 0.76, 0, 1, 1]]
# data += [["A", 0.86, 1, 0, 0]]
data += [["C", 0.93, 0, 1, 1]]
data += [["C", 0.47, 0, 1, 1]]
# data += [["A", 0.73, 1, 0, 0]]
# data += [["B", 0.89, 1, 2, 0]]

print(entropy(data, 0))

print(f"IG(y{1}) = {round(information_gain_cont(data, 1)[0], 3), information_gain_cont(data, 1)[1]}")
for i in [2, 3, 4]:
    print(f"IG(y{i}) = {round(information_gain(data, i), 3)}")

1.5
IG(y1) = (0.811, 0.385)
IG(y2) = 0.0
IG(y3) = 0.0
IG(y4) = 0.811


Como y1 e y4 têm ambos ganho de informação igual, escolhe-se por ordem alfabética y1, com corte em 0.385.